<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

<font face='georgia'>
    <h4><strong>1.  TFIDF Vectorizer & results comaparison with Sklearn:</strong></h4>

<ul>
    <li>implementing TFIDF vectorizer on a collection of text documents.</li>
    <br>
    <li>  comparison of results of your own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.</li>
    <br>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results we are adding following things to your custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sroted in alphabetical order</li>
        <li> Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
        </ol>
   </ul>

 

### Corpus

In [2]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [ ]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [ ]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [ ]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [ ]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Your custom implementation

In [3]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy
import sklearn
#This fit method return the vocab when the corpus/dataset is given input to it.
def fit(dataset):
#using set to remove duplicates from the vocab as sets dont contain duplicate values.
    unique=set()
#checking whether the input data is list or not.
    if isinstance(dataset,list):
#For each line in the dataset.
        for row in dataset:
#for each word in the line.
            for word in row.split(" "):
#setting limit >2 to remove full stops and commas.
                if len(word)<2:
                    continue
                unique.add(word)
#sorting the vocab
        unique=sorted(list(unique))
#creating a dictionary with indexing.
        u={i:j for j,i in enumerate(unique)}
        return u
    else:
        print("have to pass the list of strings")

In [4]:
#printing the vocab.
print(fit(corpus))

{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}


In [5]:
#This method ff is used to count the occurence of a word in entire corpus/dataset, used in idf method.
def ff(corpus):
    p=[]
    for i in corpus:
        l=i.split()
        k=set()
        for ele in l:
            if (i.count(ele)>1 and (i not in k) or i.count(ele)==1):
                k.add(ele)
        k=list(k)
        p=p+k
    return(p)
#This function idf returns the idf value.
def idf(corpus,i):
  return  1+(math.log((1+len(corpus))/(1+ff(corpus).count(i))))
#This method is ued to print the idf values of the feature names.
def idfvalues(corpus):
    id=[]
    for i in fit(corpus).keys():
      ele=idf(corpus,i)
      id.append(ele)
    return id

In [6]:
#IDF values of the corpus.
print(idfvalues(corpus))

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [7]:
#Here this transform method generates sparse tdidf matrix with l2 normalisation.
def transform(dataset,vocab):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # iterating each document in the dataset.
            # getting the word frequency into dictonary to compute term frequency.
            word_freq = dict(Counter(row.split()))
            # interating over every word in the document.
            for word, freq in word_freq.items():
              #removing full stops and commas                
                if len(word) < 2:
                    continue
                # check whether the word is there in vocab or not
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # storing the index of the document
                    rows.append(idx)
                    # storing the dimensions of the word
                    columns.append(col_index)
                    # storing the frequency of the word
                    td=freq/float(len(row))
                    values.append((td)*(idf(dataset,word)))
            #converting into sparse matrix.
            X= csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
            #applying l2 normalisation over the sparse matrix.
            return sklearn.preprocessing.normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
    else:
        print("you need to pass list of strings")


In [11]:
#printing the output of tranform method.
print("trasform method output.")
print(transform(corpus,fit(corpus)))
print('\n'*4)
print(transform(corpus,fit(corpus)).toarray())

  0%|          | 0/4 [00:00<?, ?it/s]

  (0, 1)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149





[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]


<font face='georgia'>
    <h4><strong> Implementation of max features functionality:</strong></h4>

<ul>
    <li> modifying our fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li> here your vocabulary is limited to only top 50 features names based on their idf values. our output will have exactly 50 columns and the number of rows will depend on the number of documents you have in your corpus.</li>
    <br>
    <li>we use a pickle file, with file name <strong>cleaned_strings</strong>. and load the corpus from this file and use it as input to our tfidf vectorizer.</li>
  
    
    
    
   </ul>

In [5]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus2 = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus2))

Number of documents in corpus =  746


In [ ]:
# Write your code here.
# Try not to hardcode any values.
# Make sure its well documented and readble with appropriate comments.

In [2]:
#This fit2 method return the vocab/feature names.
def fit2(corpus):
  #Here we are using previous fit method to get feature values and sorted them according to their idf values and gettting top 50 values.
  return dict(Counter(dict(sorted((dict(zip((fit(corpus)),(idfvalues(corpus))))).items(), key=lambda item: item[1],reverse=True))).most_common(50))

In [17]:
#printing the IDF(top 50) values with corpus
print(fit2(corpus2))

{'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.922918004572872, 'abstruse': 6.922918004572872, 'academy': 6.922918004572872, 'accents': 6.922918004572872, 'accessible': 6.922918004572872, 'acclaimed': 6.922918004572872, 'accolades': 6.922918004572872, 'accurate': 6.922918004572872, 'accurately': 6.922918004572872, 'achille': 6.922918004572872, 'ackerman': 6.922918004572872, 'actions': 6.922918004572872, 'adams': 6.922918004572872, 'add': 6.922918004572872, 'added': 6.922918004572872, 'admins': 6.922918004572872, 'admiration': 6.922918004572872, 'admitted': 6.922918004572872, 'adrift': 6.922918004572872, 'adventure': 6.922918004572872, 'aesthetically': 6.922918004572872, 'affected': 6.922918004572872, 'affleck': 6.922918004572872, 'afternoon': 6.922918004572872, 'aged': 6.922918004572872, 'ages': 6.922918004572872, 'agree': 6.922918004572872, 'agreed': 6.922918004572872, 'aimless': 6.922918004572872, 'aired': 6.922918004572872, 'akasha': 6.922918004572872, '

In [25]:
#here we are created a custom transform method similar to the previous transform method.
from sklearn.preprocessing import normalize
import sklearn
def transform2(dataset,vocab):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(tqdm(dataset)): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2:
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    td=freq/float(len(row))
                    values.append((td)*(idf2(dataset,word)))
            X= csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
            return sklearn.preprocessing.normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
    else:
        print("you need to pass list of strings")

In [26]:
#selecting a random text from the data (cleaned_strings).
print(corpus2[2])
randomtext=[corpus2[2]]


attempting artiness black white clever camera angles movie disappointed became even ridiculous acting poor plot lines almost non existent


In [27]:
#printing the output in a dense format.
print((transform2(randomtext,fit2(corpus2)).todense()))

  0%|          | 0/1 [00:00<?, ?it/s]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [28]:
print((transform2(randomtext,fit2(corpus2)).toarray()))

  0%|          | 0/1 [00:00<?, ?it/s]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [29]:
#printing the shape of the output.
print((transform(randomtext,fit2(corpus2)).todense()).shape)

  0%|          | 0/1 [00:00<?, ?it/s]

(1, 50)
